<a href="https://colab.research.google.com/github/sv4u/hashtag-generator/blob/master/HastagGeneratorDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hashtag Generator Development

TODO: insert broad description for entire file

## Import Statements

TODO: insert reasoning behind each import

In [2]:
import datetime
import requests
import json
import pandas as pd

## API Setup Functions

TODO: add broad description for this section

In [3]:
def getCreds():
	""" Get creds required for use in the applications
	
	Returns:
		dictonary: credentials needed globally
	"""

	creds = dict() # dictionary to hold everything
	creds['access_token'] = 'EAAFEzPo2nKMBAEnaqLj74ZAiXIYYZAZB8RUKFmZCZAPIVzaG7eZCHOexp5v4ZAuiehn5MSzyw8NmQMhNPeRCNKKZBa5L0WTeZCZAHMHA1wyBCHK08rY3xLsyLdHlfYvSfHtHpf1abiMdZAC55qZAv8dXfQQKYT0PDNeF9UHyXiyaCjqp0AZDZD' # access token for use with all api calls
	creds['client_id'] = '357122138610851' # client id from facebook app hashtag-generator
	creds['client_secret'] = '8a3384e6ad85e79ecea9e3b24fb8db20' # client secret from facebook app
	creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
	creds['graph_version'] = 'v8.0' # version of the api we are hitting
	creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint with domain and version
	creds['debug'] = 'no' # debug mode for api call
	creds['page_id'] = '103099641435304' # users page id
	creds['instagram_account_id'] = '17841434402398007' # users instagram account id
	creds['ig_username'] = 'getaheadtutor' # ig username

	return creds

def makeApiCall( url, endpointParams, debug = 'no' ):
	""" Request data from endpoint with params
	
	Args:
		url: string of the url endpoint to make request from
		endpointParams: dictionary keyed by the names of the url parameters
	Returns:
		object: data from the endpoint
	"""

	data = requests.get( url, endpointParams ) # make get request; gets output data from API call

	response = dict() # hold response info
	response['url'] = url # url we are hitting
	response['endpoint_params'] = endpointParams #parameters for the endpoint
	response['endpoint_params_pretty'] = json.dumps( endpointParams, indent = 4 ) # pretty print for cli
	response['json_data'] = json.loads( data.content ) # response data from the api
	response['json_data_pretty'] = json.dumps( response['json_data'], indent = 4 ) # pretty print for cli

	if ( 'yes' == debug ) : # display out response info
		displayApiCallData( response ) # display response

	return response # get and return content

def displayApiCallData( response ) :
	""" Print out to cli response from api call """

	print("\nURL: ") # title
	print(response['url']) # display url hit
	print("\nEndpoint Params: ") # title
	print(response['endpoint_params_pretty']) # display params passed to the endpoint
	print("\nResponse: ") # title
	print(response['json_data_pretty']) # make look pretty for cli

def debugAccessToken( params ) :
	""" Get info on an access token 
	
	API Endpoint:
		https://graph.facebook.com/debug_token?input_token={input-token}&access_token={valid-access-token}
	Returns:
		object: data from the endpoint
	"""

	endpointParams = dict() # parameter to send to the endpoint
	endpointParams['input_token'] = params['access_token'] # input token is the access token
	endpointParams['access_token'] = params['access_token'] # access token to get debug info on

	url = params['graph_domain'] + '/debug_token' # endpoint url

	return makeApiCall( url, endpointParams, params['debug'] ) # make the api call

def getLongLivedAccessToken( params ):
	""" Get long lived access token
	
	API Endpoint:
		https://graph.facebook.com/{graph-api-version}/oauth/access_token?grant_type=fb_exchange_token&client_id={app-id}&client_secret={app-secret}&fb_exchange_token={your-access-token}
	Returns:
		object: data from the endpoint
	"""

	endpointParams = dict() # parameter to send to the endpoint
	endpointParams['grant_type'] = 'fb_exchange_token' # tell facebook we want to exchange token
	endpointParams['client_id'] = params['client_id'] # client id from facebook app
	endpointParams['client_secret'] = params['client_secret'] # client secret from facebook app
	endpointParams['fb_exchange_token'] = params['access_token'] # access token to get exchange for a long lived token

	url = params['endpoint_base'] + 'oauth/access_token' # endpoint url

	return makeApiCall( url, endpointParams, params['debug'] ) # make the api call

def getHashtagInfo( hashtag, params ) :
	""" Get info on a hashtag
	
	API Endpoint:
		https://graph.facebook.com/{graph-api-version}/ig_hashtag_search?user_id={user-id}&q={hashtag-name}&fields={fields}
	Returns:
		object: data from the endpoint
	"""

	endpointParams = dict() # parameter to send to the endpoint
	endpointParams['user_id'] = params['instagram_account_id'] # user id making request
	endpointParams['q'] = hashtag # hashtag name
	endpointParams['fields'] = 'id,name' # fields to get back
	endpointParams['access_token'] = params['access_token'] # access token

	url = params['endpoint_base'] + 'ig_hashtag_search' # endpoint url

	return makeApiCall( url, endpointParams, params['debug'] ) # make the api call

def getHashtagMedia( params ) :
	""" Get posts for a hashtag
	
	API Endpoints:
		https://graph.facebook.com/{graph-api-version}/{ig-hashtag-id}/top_media?user_id={user-id}&fields={fields}
		https://graph.facebook.com/{graph-api-version}/{ig-hashtag-id}/recent_media?user_id={user-id}&fields={fields}
	Returns:
		object: data from the endpoint
	"""

	endpointParams = dict() # parameter to send to the endpoint
	endpointParams['user_id'] = params['instagram_account_id'] # user id making request
	endpointParams['fields'] = 'id,children,caption,comments_count,like_count,media_type,media_url,permalink' # fields to get back
	endpointParams['access_token'] = params['access_token'] # access token

	url = params['endpoint_base'] + params['hashtag_id'] + '/' + params['type'] # endpoint url

	return makeApiCall( url, endpointParams, params['debug'] ) # make the api call

def getHashtagMediaAll( hashtag_id, params ):
	"""
	Get top and recent hashtag media for hashtag_id
	"""
	
	params = params.copy() #create local version of cred params

	# params['hashtag_id'] = '17841593875074036' #provides target hashtag id
	params['hashtag_id'] = hashtag_id #provides target hashtag id
	params['type'] = 'top_media' #sets media type to top
	top_response = getHashtagMedia(params) #function call that returns post info

	params['hashtag_id'] = hashtag_id #provides target hashtag id
	params['type'] = 'recent_media' #sets media type to recent
	recent_response = getHashtagMedia(params) #function call that returns post info

	return top_response, recent_response

def getMediaCommentCount ( top_response, recent_response ) : 
	""" 
		Gets comment counts from recent and top posts, and compiles data into a list of tuples
		containing the media id and its comment count
	"""
	comment_counts = [] #creates a list that will contain each top post's comment count
	comment_counts = [post['comments_count'] for post in top_response['json_data']['data']]  #appends each top post's comment count to the list
	post_id = [] #creates a list that will contain each top post's id
	post_id = [post['id'] for post in top_response['json_data']['data']] #appends each top post id to the list
	post_id_and_comment_counts = list(zip(post_id, comment_counts)) #creates a tuple that combines top post id and comment count

	recent_comment_counts = [] #creates a list that will contain each recent post's comment count
	recent_comment_counts = [post['comments_count'] for post in recent_response['json_data']['data']] #appends each recent post's comment count to the list
	recent_post_id = [] #creates a list that will contain each recent post's id
	recent_post_id = [post['id'] for post in recent_response['json_data']['data']] #appends each recent post id to the list
	recent_post_id_and_comment_counts = list(zip(recent_post_id, recent_comment_counts)) #creates a tuple that combines recent post id and comment count
 
	post_id_and_comment_counts += recent_post_id_and_comment_counts #appends recent comment counts to top comment counts3

	return post_id_and_comment_counts

def getMediaLikeCount ( top_response, recent_response ) : 
	""" 
		Gets like counts from recent and top posts, and compiles data into a list of tuples containing the media
		id and its number of likes.
	"""
	like_counts = [] #creates a list that will contain each top post's like count
	like_counts = [post['like_count'] for post in top_response['json_data']['data']]  #appends each top post's like count to the list
	post_id = [] #creates a list that will contain each top post's id
	post_id = [post['id'] for post in top_response['json_data']['data']] #appends each top post id to the list
	post_id_and_like_counts = list(zip(post_id, like_counts)) #creates a tuple that combines top post id and like count
	
	recent_like_counts = [] #creates a list that will contain each recent post's like count
	recent_like_counts = [post['like_count'] for post in recent_response['json_data']['data']] #appends each recent post's like count to the list
	recent_post_id = [] #creates a list that will contain each recent post's id
	recent_post_id = [post['id'] for post in recent_response['json_data']['data']] #appends each recent post id to the list
	recent_post_id_and_like_counts = list(zip(recent_post_id, recent_like_counts)) #creates a tuple that combines recent post id and like count
 
	post_id_and_like_counts += recent_post_id_and_like_counts #appends recent like counts to top like counts
	
	return post_id_and_like_counts

def getMediaCaptions ( top_response, recent_response ) :
	"""
		Creates a list of tuples containing media ids and their captions
	"""
	captions = [] #creates a list that will contain each top post's caption
	captions = [post['caption'] for post in top_response['json_data']['data']] #appends each top post's caption to the list
	post_id = [] #creates a list that will contain each top post's id
	post_id = [post['id'] for post in top_response['json_data']['data']] #appends each top post id to the list
	post_id_and_captions = list(zip(post_id,captions)) #creates a tuple that combines top post id and captions
	
	recent_captions = [] #creates a list that will contain each recent post's caption
	recent_captions = [post['caption'] for post in recent_response['json_data']['data']] #appends each recent post's like caption to the list
	recent_post_id = [] #creates a list that will contain each recent post's id
	recent_post_id = [post['id'] for post in recent_response['json_data']['data']] #appends each recent post id to the list
	recent_post_id_and_captions = list(zip(recent_post_id, recent_captions)) #creates a tuple that combines recent post id and captions
 
	post_id_and_captions += recent_post_id_and_captions #appends recent captions to top captions
	
	return post_id_and_captions

def getRelatedHashtags ( captions_and_ids ) : 
	"""
		Creates a list of tuples that each contain a media id and a list of the media's related hashtags.
	"""
	# captions_and_ids = getMediaCaptions() #function call that returns a list of media ids and their captions.
	ids, captions = zip(*captions_and_ids) #separates the ids and captions into 2 lists
	split_captions = [caption.split() for caption in captions] #separates each caption into a list of words
	related_hashtags = [] #declaration of a list that will store lists of hashtags, wherein each inner list represents a single post 
	for caption in split_captions: #iterates through each caption in the initial list of captions
		caption_hashtags = [] #creates a new inner list for each caption 
		for item in caption: #iterates through each item in the caption 
			if item[0] == '#': #checks if the item's first character is a hashtag
				caption_hashtags.append(item) #adds the item to the inner list if so
		related_hashtags.append(caption_hashtags) #adds the inner list to the outter list 
	related_hashtags = list(zip(ids, related_hashtags)) #creates a list of tuples that each contain the post id and a list of its related hashtags
	
	return related_hashtags

def compilePostData ( like_counts, comment_counts, media_captions, media_hashtags ) :
	"""
		Creates a list of dictionaries that each represent a single ID and contain related hashtags and pertinent metrics
	"""
	# like_counts = getMediaLikeCount()
	# comment_counts = getMediaCommentCount()
	# media_captions = getMediaCaptions()
	# media_hashtags = getRelatedHashtags()
 
	ids, likes = zip(*like_counts) #separates ids and likes into two lists
	ids, comments = zip(*comment_counts) #separates ids and comments into two lists
	ids, captions = zip(*media_captions) #separates ids and captions into two lists
	ids, hashtags = zip(*media_hashtags) #separates ids and list of hashtags into two lists (creates a separate nested list of hashtags)
	
	list_of_dicts = [] #creates list that will hold dictionaries that each represent a single post
	index = 0 #counter variable for iterating through captions, hashtags, likes, and comments
	for id in ids: 
		id_dict = {} #creates a new dict for each media id (post) 
		id_dict['id'] = id #appends the current post id to the current dict
		id_dict['caption'] = captions[index] #appends the current caption to the current dict
		id_dict['hashtags'] = hashtags[index] #appends the current list of hashtags to the current dict
		id_dict['likes'] = likes[index] #appends the current like count to the current dict
		id_dict['comments'] = comments[index] #appends the current comment count to the current dict
		list_of_dicts.append(id_dict) #appends the current dict to the big list of dicts
		index += 1 #increments counter variable
	return list_of_dicts

def getLikeTotals ( related_hashtags, list_of_dictionaries ) : 
	"""
		Creates a nested list containing like totals for each individual hashtag on a piece of related media
	"""
	# related_hashtags = getRelatedHashtags()
	# list_of_dictionaries = compilePostData()
	
	ids, lists_of_hashtags = zip(*related_hashtags)  #separates ids and list of hashtags (creates a separate nested list of hashtags)
	aggregated_like_counts = [] #a list that will contain lists each representing a single post, and whose items each rep a hashtag's like count
	for list_ in lists_of_hashtags: #iterates through each smaller list in the nested list of hashtags
		like_counts = [] #creates a new list for each smaller list in nested list of hashtags
		for hashtag in list_: #iterates through each individual hashtag in the smaller list 
			like_count = 0 #sets like count to 0
			for dictionary in list_of_dictionaries: #iterates through each dict in list of dicts (each post)
				if hashtag in dictionary['hashtags']: #checks if the current hashtag is in the 'hashtags' list of the dict 
					like_count += dictionary['likes'] #if so, adds the respective likes to like count
			like_counts.append(like_count) #once iterated through all dicts (posts), adds final like count for the current hashtag to the like counts list
		aggregated_like_counts.append(like_counts) #once iterated through all hashtags in each smaller list, adds like counts list to aggreagated list
	return aggregated_like_counts
			
def getCommentTotals ( related_hashtags, list_of_dictionaries ) :  
	"""
		Creates a nested list containing comment totals for each individual hashtag on a piece of related media
	"""
	# related_hashtags = getRelatedHashtags()
	# list_of_dictionaries = compilePostData()
	
	ids, lists_of_hashtags = zip(*related_hashtags) #separates ids and list of hashtags (creates a separate nested list of hashtags)
	aggregated_comment_counts = [] #a list that will contain lists each representing a single post, and whose items each rep a hashtag's comment count
	for list_ in lists_of_hashtags: #iterates through each smaller list in the nested list of hashtags
		comment_counts = [] #creates a new list for each smaller list in nested list of hashtags
		for hashtag in list_: #iterates through each individual hashtag in the smaller list 
			comment_count = 0 #sets comment count to 0
			for dictionary in list_of_dictionaries: #iterates through each dict in list of dicts (each post)
				if hashtag in dictionary['hashtags']: #checks if the current hashtag is in the 'hashtags' list of the dict 
					comment_count += dictionary['comments'] #if so, adds the respective comments to like count
			comment_counts.append(comment_count) #once iterated through all dicts (posts), adds final comment count for the current hashtag to the comment counts list
		aggregated_comment_counts.append(comment_counts)  #once iterated through all hashtags in each smaller list, adds comment counts list to aggreagated list
	return aggregated_comment_counts

def getNumberOfPosts ( related_hashtags, list_of_dictionaries ) :  
	"""
		Creates a nested list containing number of posts for each individual hashtag on a piece of related media
	"""
	# related_hashtags = getRelatedHashtags()
	# list_of_dictionaries = compilePostData()
	
	ids, lists_of_hashtags = zip(*related_hashtags) #separates ids and list of hashtags (creates a separate nested list of hashtags)
	aggregated_post_counts = []  #a list that will contain lists each representing a single post, and whose items each rep a hashtag's post count
	for list_ in lists_of_hashtags: #iterates through each smaller list in the nested list of hashtags
		post_counts = [] #creates a new list for each smaller list in nested list of hashtags
		for hashtag in list_: #iterates through each individual hashtag in the smaller list 
			post_count = 0 #sets post count to 0
			for dictionary in list_of_dictionaries: #iterates through each dict in list of dicts (each post)
				if hashtag in dictionary['hashtags']: #checks if the current hashtag is in the 'hashtags' list of the dict 
					post_count += 1 #if so, increments post count
			post_counts.append(post_count) #once iterated through all dicts (posts), adds final post count for the current hashtag to the post counts list
		aggregated_post_counts.append(post_counts)  #once iterated through all hashtags in each smaller list, adds post counts list to aggreagated list
	return aggregated_post_counts
	
def compileHashtagData ( related_hashtags, list_of_dictionaries, number_of_posts, total_likes, total_comments) :
	"""
		Creates a nested dict with each hashtag and their respective values.			
	"""
	# related_hashtags = getRelatedHashtags()
	# list_of_dictionaries = compilePostData()
	# number_of_posts = getNumberOfPosts()
	# total_likes = getLikeTotals()
	# total_comments = getCommentTotals()
	
	ids, lists_of_hashtags = zip(*related_hashtags) #separates ids and list of hashtags (creates a separate nested list of hashtags)
	hashtag_info = {} #creates a dict that will contain dicts each representing a hashtag's metrics

	outer_index = 0 #sets counter variable that will iterate through each list in the nested lists for total likes, comments, and posts
	for list_ in lists_of_hashtags: #iterates through each smaller list in the nested list of hashtags
		inner_index = 0 #sets counter variable that will iterate through each item for the current list
		for hashtag in list_: #iterates through each individual hashtag in the smaller list 
			if hashtag not in hashtag_info: #makes sure hashtag is not already added to the hashtag dict
				hashtag_info['{}'.format(hashtag)] = {} #creates a new dict for each new hashtag that is not already in nested dict
				hashtag_info['{}'.format(hashtag)]['total_likes'] = total_likes[outer_index][inner_index] #adds total likes to new dict
				hashtag_info['{}'.format(hashtag)]['total_comments'] = total_comments[outer_index][inner_index] #adds total comments to new dict
				hashtag_info['{}'.format(hashtag)]['number_of_posts'] = number_of_posts[outer_index][inner_index] #adds post count to new dict
			inner_index += 1 #increments inner counter variable (next item in smaller list)
		outer_index += 1 #increments outer counter variable (new list in nested list)
	return hashtag_info



### Example API Calls

TODO: explain what this example is doing

In [14]:
params = getCreds()

In [15]:
hashtag_info = getHashtagInfo( 'blogger', params )
hashtag_id = hashtag_info['json_data']['data'][0]['id']

In [16]:
top_response, recent_response = getHashtagMediaAll(hashtag_id, params)

In [17]:
recent_response

{'url': 'https://graph.facebook.com/v8.0/17843713579011610/recent_media',
 'endpoint_params': {'user_id': '17841434402398007',
  'fields': 'id,children,caption,comments_count,like_count,media_type,media_url,permalink',
  'access_token': 'EAAFEzPo2nKMBAEnaqLj74ZAiXIYYZAZB8RUKFmZCZAPIVzaG7eZCHOexp5v4ZAuiehn5MSzyw8NmQMhNPeRCNKKZBa5L0WTeZCZAHMHA1wyBCHK08rY3xLsyLdHlfYvSfHtHpf1abiMdZAC55qZAv8dXfQQKYT0PDNeF9UHyXiyaCjqp0AZDZD'},
 'endpoint_params_pretty': '{\n    "user_id": "17841434402398007",\n    "fields": "id,children,caption,comments_count,like_count,media_type,media_url,permalink",\n    "access_token": "EAAFEzPo2nKMBAEnaqLj74ZAiXIYYZAZB8RUKFmZCZAPIVzaG7eZCHOexp5v4ZAuiehn5MSzyw8NmQMhNPeRCNKKZBa5L0WTeZCZAHMHA1wyBCHK08rY3xLsyLdHlfYvSfHtHpf1abiMdZAC55qZAv8dXfQQKYT0PDNeF9UHyXiyaCjqp0AZDZD"\n}',
 'json_data': {'data': [{'id': '17851560182259849',
    'caption': 'Vamos dar um rolê 🤣 🐼🥰🤭🚗\n.\n \n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#bebeblogueiro #nenemdamamae 

In [18]:
comment_counts = getMediaCommentCount(top_response, recent_response)

In [19]:
like_counts = getMediaLikeCount(top_response, recent_response)

In [20]:
captions_and_ids = getMediaCaptions(top_response, recent_response)

In [21]:
related_hashtags = getRelatedHashtags(captions_and_ids)

In [22]:
list_of_dictionaries = compilePostData(like_counts, comment_counts, captions_and_ids, related_hashtags)

In [23]:
list_of_dictionaries

[{'id': '17879179540756647',
  'caption': 'Dear Chicago, you will forever be my favorite city & have a piece of my heart!  Happy to be home for a few days to enjoy the weather and destress! What’s your favorite city to visit?',
  'hashtags': [],
  'likes': 220,
  'comments': 28},
 {'id': '17866854095002230',
  'caption': 'Post olmasına karar verildi 🦋📸\n.\n.\n.\n.\n.\n.\n.\n#throwback #hello #likeforlikes #instagram #peaceful #instagood #follow4followback #lookatme  #sunnyday #goodday #funday #today  #followforfollowback #blog #blogger #smile #instalike #lookatme  #türkiye #fashion  #women  #friday #goodnight #rose #travel #photography #summervibes  #sea #selfie #kiss #sunday',
  'hashtags': ['#throwback',
   '#hello',
   '#likeforlikes',
   '#instagram',
   '#peaceful',
   '#instagood',
   '#follow4followback',
   '#lookatme',
   '#sunnyday',
   '#goodday',
   '#funday',
   '#today',
   '#followforfollowback',
   '#blog',
   '#blogger',
   '#smile',
   '#instalike',
   '#lookatme',
  

In [24]:
number_of_posts = getNumberOfPosts(related_hashtags, list_of_dictionaries)

In [25]:
total_likes = getLikeTotals(related_hashtags, list_of_dictionaries)

In [26]:
total_comments = getCommentTotals(related_hashtags, list_of_dictionaries)

In [27]:
hashtag_data = compileHashtagData(related_hashtags, list_of_dictionaries, number_of_posts, total_likes, total_comments)

In [28]:
hashtag_data

{'#throwback': {'total_likes': 255,
  'total_comments': 16,
  'number_of_posts': 1},
 '#hello': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#likeforlikes': {'total_likes': 705,
  'total_comments': 91,
  'number_of_posts': 5},
 '#instagram': {'total_likes': 255,
  'total_comments': 16,
  'number_of_posts': 1},
 '#peaceful': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#instagood': {'total_likes': 1253,
  'total_comments': 149,
  'number_of_posts': 9},
 '#follow4followback': {'total_likes': 438,
  'total_comments': 89,
  'number_of_posts': 2},
 '#lookatme': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#sunnyday': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#goodday': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#funday': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#today': {'total_likes': 255, 'total_comments': 16, 'number_of_posts': 1},
 '#followfo

In [29]:
df = pd.DataFrame(hashtag_data).T.sort_values('total_comments')

In [30]:
df

,total_likes,total_comments,number_of_posts
#marketing,1,0,1
#minimodelo,0,0,1
#minimodel,0,0,1
#blogueirinha,0,0,1
#amorincondicional,0,0,1
...,...,...,...
#lookdodia,2282,97,4
#happy,431,107,5
#fashion,3667,133,13
#instagood,1253,149,9


In [31]:
df['avg_engagement'] = (df['total_likes'] + df['total_comments']) / df['number_of_posts']

In [32]:
df.sort_values('number_of_posts', ascending=False).head(20)

,total_likes,total_comments,number_of_posts,avg_engagement
#blogger,6997,349,34,216.058824
#fashion,3667,133,13,292.307692
#instagood,1253,149,9,155.777778
#love,2095,89,7,312.000000
#photooftheday,699,54,7,107.571429
#style,935,84,7,145.571429
#fashionblogger,833,33,7,123.714286
#photography,379,22,6,66.833333
#influencer,396,30,6,71.000000
#model,398,16,6,69.000000


In [33]:
df.describe()

,total_likes,total_comments,number_of_posts,avg_engagement
count,513.000000,513.000000,513.000000,513.000000
mean,254.719298,14.003899,1.467836,173.884961
std,585.093261,26.706375,1.841216,348.857837
min,0.000000,0.000000,1.000000,0.000000
25%,1.000000,0.000000,1.000000,1.000000
50%,10.000000,1.000000,1.000000,5.000000
75%,255.000000,16.000000,1.000000,256.000000
max,6997.000000,349.000000,34.000000,2213.000000


### Another Example of API Calls

TODO: explain what this example is doing and how it works

In [ ]:
params = getCreds()
params['hashtag_id'] = '17841563278115808'
params['type'] = 'top_media'
top_response = getHashtagMedia(params)
params['type'] = 'recent_media'
recent_response = getHashtagMedia(params)

NameError: ignored

In [ ]:
top_response

NameError: ignored

In [ ]:
recent_response